In [2]:
import random
import math
import jdc

In [6]:
class Graph(object):
    def __init__(self, distances: list, rank: int):
        self.distances = distances
        self.rank = rank
        self.pheromone = [[1 / (rank * rank) for j in range(rank)] for i in range(rank)]


In [7]:
class ACS(object):
    def __init__(self, ant_count: int, generations: int, alpha: float, beta: float, rho: float, q: int):
        self.generations = generations
        self.ant_count = ant_count
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.Q = q        

In [8]:
%%add_to ACS

def _update_pheromone(self, graph: Graph, ants: list):
        for i, row in enumerate(graph.pheromone):
            for j, col in enumerate(row):
                graph.pheromone[i][j] *= self.rho
                for ant in ants:
                    graph.pheromone[i][j] += ant.pheromone_delta[i][j]

In [9]:
%%add_to ACS

def solve(self, graph: Graph):
        best_cost = float('inf')
        best_solution = []
        for gen in range(self.generations):
            ants = [_Ant(self, graph) for i in range(self.ant_count)]
            for ant in ants:
                for i in range(graph.rank - 1):
                    ant._select_next()
                ant.total_cost += graph.distances[ant.visited_nodes[-1]][ant.visited_nodes[0]]
                if ant.total_cost < best_cost:
                    best_cost = ant.total_cost
                    best_solution = [] + ant.visited_nodes
                ant._update_pheromone_delta()
            self._update_pheromone(graph, ants)
        return best_solution, best_cost

In [ ]:
class _Ant(object):
    def __init__(self, acs: ACS, graph: Graph):
        self.colony = acs
        self.graph = graph
        self.total_cost = 0.0
        self.visited_nodes = []  # visited nodes list
        self.pheromone_delta = []  # the local increase of pheromone
        self.allowed_nodes = [i for i in range(graph.rank)]  # nodes which are allowed for the next selection
        self.eta = [[0 if i == j else 1 / graph.distances[i][j] for j in range(graph.rank)] for i in
                    range(graph.rank)]  # heuristic information
        start_node = random.randint(0, graph.rank - 1)  # start from any node
        self.visited_nodes.append(start_node)
        self.current_node = start_node
        self.allowed_nodes.remove(start_node)


In [10]:
%%add_to _Ant
def _select_next(self):
    denominator = 0
    for i in self.allowed_nodes:
        denominator += self.graph.pheromone[self.current_node][i] ** self.colony.alpha * \
            self.eta[self.current_node][i] ** self.colony.beta

    probabilities = [0 for i in range(self.graph.rank)]  # probabilities for moving to a node in the next step
    for i in range(self.graph.rank):
        try:
            self.allowed_nodes.index(i)  # test if allowed list contains i
            probabilities[i] = self.graph.pheromone[self.current_node][i] ** self.colony.alpha * \
                self.eta[self.current_node][i] ** self.colony.beta / denominator
        except ValueError:
            pass  # do nothing

    # select next node by probability roulette
    selected_node = 0
    rand = random.random()
    for i, probability in enumerate(probabilities):
        rand -= probability
        if rand <= 0:
            selected_node = i
            break

    self.allowed_nodes.remove(selected_node)
    self.visited_nodes.append(selected_node)
    self.total_cost += self.graph.distances[self.current_node][selected_node]
    self.current_node = selected_node

In [ ]:
%%add_to _Ant
def _update_pheromone_delta(self):
        self.pheromone_delta = [[0 for j in range(self.graph.rank)] for i in range(self.graph.rank)]
        for _ in range(1, len(self.visited_nodes)):
            i = self.visited_nodes[_ - 1]
            j = self.visited_nodes[_]
            self.pheromone_delta[i][j] = self.colony.Q / self.total_cost

In [11]:
def distance(city1: dict, city2: dict):
    return math.sqrt((city1['x'] - city2['x']) ** 2 + (city1['y'] - city2['y']) ** 2)

In [12]:
ant_count = 10
generations = 100
alpha = 1.0
beta = 10.0
rho = 0.5
q = 10

In [13]:
cities = []
points = []
with open('./data/cities-v1.txt') as f:
    for line in f.readlines():
        city = line.split(' ')
        cities.append(dict(index=int(city[0]), x=int(city[1]), y=int(city[2])))
        points.append((int(city[1]), int(city[2])))
distances = []
number_of_cities = len(cities)
for i in range(number_of_cities):
    row = []
    for j in range(number_of_cities):
        row.append(distance(cities[i], cities[j]))
    distances.append(row)

acs = ACS(ant_count, generations, alpha, beta, rho, q)
graph = Graph(distances, number_of_cities)
path, cost = acs.solve(graph)
print('cost: {}, path: {}'.format(cost, path))

cost: 15601.919532918737, path: [0, 14, 13, 11, 12, 10, 22, 15, 4, 5, 6, 1, 3, 7, 8, 9, 2, 17, 16, 18, 23, 24, 19, 20, 21, 25, 27, 26, 29, 30, 28]
